In [164]:
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import time

In [165]:
# 保育園データの読み込み
df = pd.read_csv("hoikuen.csv")

In [166]:
# 現在の日付を取得
today = datetime.date.today()

# 次の週の月曜日を取得
next_monday = today + datetime.timedelta(days=(7-today.weekday()))

# 2023年の日本の祝日のリストを作成
holidays = {
    datetime.date(2023, 1, 1): "元日",
    datetime.date(2023, 1, 9): "成人の日",
    datetime.date(2023, 2, 11): "建国記念日",
    datetime.date(2023, 2, 23): "天皇誕生日",
    datetime.date(2023, 3, 21): "春分の日",
    datetime.date(2023, 4, 29): "昭和の日",
    datetime.date(2023, 5, 3): "憲法記念日",
    datetime.date(2023, 5, 4): "みどりの日",
    datetime.date(2023, 5, 5): "こどもの日",
    datetime.date(2023, 7, 17): "海の日",
    datetime.date(2023, 8, 11): "山の日",
    datetime.date(2023, 9, 18): "敬老の日",
    datetime.date(2023, 9, 23): "秋分の日",
    datetime.date(2023, 10, 9): "体育の日",
    datetime.date(2023, 11, 3): "文化の日",
    datetime.date(2023, 11, 23): "勤労感謝の日",
    datetime.date(2023, 12, 23): "天皇誕生日",
}


# 次の月曜日が日本の祝日の場合は、次の平日を計算
if next_monday in holidays:
    next_monday = next_monday + datetime.timedelta(days=1)
print(next_monday)

2023-03-20


In [167]:
# 事務所最寄り駅と徒歩時間の入力
office_station = "東京"
office_walk_Time = 10
# 検索日の設定
sarch_year = f'{next_monday.year:04}'
sarch_month = f'{next_monday.month:02}'
sarch_day = f'{next_monday.day:02}'

In [168]:
# 保育園事に作成する変数

for i, row in df.iterrows():
    
    hoikuen_end = pd.to_datetime(row["閉所時間"])

    # 閉所10分前に保育園到着換算
    arrival_time = hoikuen_end - datetime.timedelta(minutes=10) - datetime.timedelta(minutes=(int(row["walk_time"])))

    sarch_hour = f'{arrival_time.hour:02}'
    sarch_minute = arrival_time.minute
    sarch_m1 = str(sarch_minute // 10)
    sarch_m2 = str(sarch_minute % 10)

    # yahoo路線図
    route_url = f"https://transit.yahoo.co.jp/search/result?from={office_station}&to={row['nearlest_station']}&fromgid=&togid=&flatlon=&tlatlon=&via=&viacode=&y={sarch_year}&m={sarch_month}&d={sarch_day}&hh={sarch_hour}&m1={sarch_m1}&m2={sarch_m2}&type=4&ticket=ic&expkind=1&userpass=1&ws=3&s=0"
    # print(route_url)

    #Requestsを利用してWebページを取得する
    route_response = requests.get(route_url)
    # BeautifulSoupを利用してWebページを解析する
    route_soup = BeautifulSoup(route_response.text, 'html.parser')
    # 経路の詳細を取得(1つ目を指定する)
    route_detail = route_soup.find("div",class_ = "routeDetail")
    # 出発発射時間の取得(詳細情報の1つ目の時間を取得) 
    staion_departure_time = route_detail.find("ul",class_ = "time").get_text()
    # 駅の出発時間を時刻に変換
    staion_departure_time = datetime.datetime.strptime(staion_departure_time, '%H:%M')
    # 徒歩時間を引いて、オフィスの出発時間に変換
    office_departure_time = staion_departure_time - datetime.timedelta(minutes=office_walk_Time)
    
    df.loc[i,"office_departure_time"] = office_departure_time.time()
    df.loc[i,"route_url"] = route_url

    print(office_departure_time.time())
    # df["出発時間"] = office_departure_time.time()
    # time.sleep(1)


17:06:00
17:13:00
17:02:00
17:04:00
17:04:00
20:52:00
18:44:00
19:48:00
19:52:00
20:26:00
17:54:00
19:48:00
19:24:00
18:47:00
06:55:00


In [169]:
df.head()

,区分,名称,住所,0歳定員,1歳定員,2歳定員,3歳定員,4歳定員,5歳定員,定員,...,閉所時間,延長保育時間（開始）,延長保育時間（終了）,latitude,longitude,latlng,nearlest_station,walk_time,office_departure_time,route_url
0,認可保育園,南奥沢☆,東京都世田谷区奥沢1−2−13,NaN,14.0,14.0,17.0,17.0,17.0,79,...,18:15:00,18:15:00,20:15:00,35.599453,139.680111,"35.5994527,139.6801106",石川台,10,17:06:00,https://transit.yahoo.co.jp/search/result?from...
1,認可保育園,奥沢☆,東京都世田谷区奥沢2−3−11,10.0,10.0,11.0,17.0,17.0,18.0,83,...,18:15:00,18:15:00,20:15:00,35.605270,139.675704,"35.6052698,139.6757044",緑が丘（東京都）,5,17:13:00,https://transit.yahoo.co.jp/search/result?from...
2,認可保育園,中町※1,東京都世田谷区中町3−27−2,11.0,16.0,18.0,19.0,20.0,21.0,105,...,18:15:00,18:15:00,20:15:00,35.615462,139.642539,"35.6154624,139.6425391",上野毛,8,17:02:00,https://transit.yahoo.co.jp/search/result?from...
3,認可保育園,玉川,東京都世田谷区玉川4−16−6,NaN,15.0,18.0,20.0,22.0,22.0,97,...,18:15:00,18:15:00,20:15:00,35.617734,139.624131,"35.6177344,139.6241309",二子玉川,12,17:04:00,https://transit.yahoo.co.jp/search/result?from...
4,認可保育園,上用賀☆,東京都世田谷区上用賀4−2−10,9.0,14.0,15.0,21.0,22.0,22.0,103,...,18:15:00,18:15:00,20:15:00,35.634891,139.630393,"35.6348913,139.6303933",用賀,15,17:04:00,https://transit.yahoo.co.jp/search/result?from...
